# Modeling

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('modeling_data.csv')

In [3]:
data = data.drop(columns=['Unnamed: 0','log_date','log_time'])

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 15 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   fare_amount      1000000 non-null  float64
 1   passenger_count  1000000 non-null  int64  
 2   log_hour         1000000 non-null  int64  
 3   log_min          1000000 non-null  int64  
 4   log_year         1000000 non-null  int64  
 5   log_month        1000000 non-null  int64  
 6   log_day          1000000 non-null  int64  
 7   7to15            1000000 non-null  int64  
 8   16to19           1000000 non-null  int64  
 9   20to6            1000000 non-null  int64  
 10  dayofweek        1000000 non-null  int64  
 11  weekday          1000000 non-null  int64  
 12  is_extracharge   1000000 non-null  int64  
 13  taxi_amount      1000000 non-null  int64  
 14  dist             1000000 non-null  float64
dtypes: float64(2), int64(13)
memory usage: 114.4 MB


In [5]:
x_train = data.iloc[:,data.columns!='fare_amount']
y_train = data['fare_amount'].values

In [6]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [7]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train,y_train,test_size=0.2,random_state=10)

In [8]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(800000, 14)
(200000, 14)
(800000,)
(200000,)


In [9]:
y_train

array([ 7.3 ,  7.7 ,  8.  , ..., 34.83, 25.  , 10.9 ])

lgbm

In [38]:
! pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 1.8 MB/s eta 0:00:00


In [11]:
import lightgbm as lgbm

In [12]:
from sklearn.model_selection import RandomizedSearchCV

In [13]:
train_set = lgbm.Dataset(x_train, y_train, silent=True)
train_set

In [16]:
lgb = lgbm.LGBMRegressor()
parameters = {'num_leaves':[20,40,60,80,100], 'min_child_samples':[5,10,15], 'max_depth':[-1,5,10,20],'learning_rate':[0.05,0.1,0.2],'reg_alpha':[0,0.01,0.03]}


In [17]:
clf = RandomizedSearchCV(lgb,parameters,n_iter=20)
clf.fit(X=x_train, y=y_train)
print(clf.best_params_)

{'reg_alpha': 0, 'num_leaves': 80, 'min_child_samples': 15, 'max_depth': 10, 'learning_rate': 0.05}


In [18]:
params ={'reg_alpha': 0, 'num_leaves': 80, 'min_child_samples': 15, 'max_depth': 10, 'learning_rate': 0.05}

In [29]:
model_lgbm = lgbm.train(params=params, train_set = train_set, num_boost_round=300)

C:\Users\jisuj\anaconda3\lib\site-packages\lightgbm\basic.py:1491: UserWarning: 'silent' argument is deprecated and will be removed in a future release of LightGBM. Pass 'verbose' parameter via 'params' instead.
  _log_warning("'silent' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.051750 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 420
[LightGBM] [Info] Number of data points in the train set: 800000, number of used features: 14
[LightGBM] [Info] Start training from score 11.335730


In [30]:
pred_test_y = model_lgbm.predict(x_valid, num_iteration = model_lgbm.best_iteration)

In [31]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(pred_test_y, y_valid)

2.85273410961514

0.001 낮아짐

randomforest

In [10]:
from sklearn.ensemble import RandomForestRegressor


In [11]:
F

In [12]:
param_grid=[{'n_estimators':[5,10,15],'max_depth' : [6, 8, 10, 12]}]

In [16]:
rf_clf = RandomForestRegressor(random_state = 0, n_jobs = -1)

In [17]:
grid_cv = GridSearchCV(rf_clf, param_grid = param_grid, cv = 3, n_jobs = -1)

In [18]:
grid_cv.fit(x_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestRegressor(n_jobs=-1, random_state=0),
             n_jobs=-1,
             param_grid=[{'max_depth': [6, 8, 10, 12],
                          'n_estimators': [5, 10, 15]}])

In [19]:
print('최적 하이퍼 파라미터: ', grid_cv.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv.best_score_))

최적 하이퍼 파라미터:  {'max_depth': 8, 'n_estimators': 15}
최고 예측 정확도: 0.7073


In [21]:
rf_clf1 = RandomForestRegressor(max_depth= 8, n_estimators =  15,
                                random_state = 0,
                                n_jobs = -1)
rf_clf1.fit(x_train, y_train)
pred = rf_clf1.predict(x_valid)

In [22]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(pred, y_valid)

2.8781751906645123

In [13]:
model_rf= RandomForestRegressor(max_depth=10)


In [14]:
model_rf.fit(x_train, y_train)

RandomForestRegressor(max_depth=10)

In [15]:
y_pred_rf = model_rf.predict(x_valid)

In [16]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(y_pred_rf, y_valid)

2.8582408185797505